In [ ]:
library(data.table)
library(VariantAnnotation)
library(gwasvcf)
library(magrittr)
library(GenomicRanges)
library(rtracklayer)
library(coloc)
library(dplyr)
#library(httr)
#library(jsonlite)
library(parallel)

In [2]:
data.path <- "/lustre/groups/itg/teams/zeggini/projects/child_diabesity/analysis/methyl/fastQTL/Results/peer20_PCA5/Results/"
working_dir = "/lustre/groups/itg/teams/zeggini/projects/child_diabesity/Coloc_pipeline/Coloc"
setwd(working_dir)


In [59]:
source("coloc/Coloc_helper_functions.R")

# Load the reference datasets

In [4]:
# These are UKBB variant annotations in hg19 coordinates
variant_ann <- fread(file.path(working_dir, "ReferenceData/variants.tsv"), data.table=FALSE)


In [5]:
# This is to annotate fastqtl output with genomic positions. This has to be the same genotype used in the QTL analysis
# The best thing is to only supply the first 5 columns of the VCF file, withoiut the header, in a txt file. In this case I called this file SNP_POS.txt
PATH_IN_G = "/lustre/groups/itg/teams/zeggini/projects/child_diabesity/data/CleanGenotypes"
SNP_POS <- fread(file.path(PATH_IN_G, "SNP_POS.txt"))



In [6]:
# For mQTLs
# Available annotations here /lustre/groups/itg/teams/zeggini/projects/fungen-oa/analyses/methylqq2/annotations/
m_anno <- get_m_anno_epic(m_anno = "EPIC.hg19.manifest.tsv.gz")


In [ ]:
# For eQTLs. This is the bed file used in the fastqtl analysis
e_path <- "/lustre/groups/itg/teams/zeggini/projects/child_diabesity/analysis/rnaseq/PrepareData"
e_anno <- fread(file.path(e_path, "geneTSS_mostAbundTxInAT_hg19_GeneExp_TMMNorm_INT_fastqtl.bed"), data.frame=FALSE)
e_anno <- e_anno[,1:4]


# Get top associations for the GWAScatalog

In [7]:
library(gwasrapidd)


Attaching package: ‘gwasrapidd’


The following object is masked from ‘package:dplyr’:

    n




In [60]:
# Set GWAScatalog ID
GWAS_ID <- "GCST009004"


In [61]:
#####################################################
########## GET SAMPLE SIZE FROM THE GWAS CATALOG
#####################################################
# This bit of code is going to extract the sample size from the GWAS catalog. 
# If you already know the sample size, just make a variable "GWAS_n" with the total sample size, for a quantitative trait, or the number of cases and controls, if a case-control study
GWAS_n <- Get_sampleSize_GWAScatalog(GWAS_ID)
print(paste0("sample size is ", GWAS_n))

[1] "sample size is 806834"


In [62]:
######################################
###### EXTRACT TOP SNP INFO FROM THE GWAS CATALOG
######################################
    
# Function to extract top hits information from the GWAScatalog. hg38
# If you use a GWAS that is not on the GWAS catalog, you need a data.frame with the columns
# variant_id: this is rsID
# chromosome_name: chromosome name without the chr prefix
# chromosome_position: chromosome position
# pvalue: pvalue
# if(!exists("GWAS_associations_df")){
GWAS_associations_df <- get_GWAScatalog_topHits(GWAS_ID)
# }
    

GWAS_associations_df <- GWAS_associations_df %>%
                            dplyr::filter(!is.na(chromosome_position),
                                          chromosome_name != "X",
                                          pvalue < 5e-8) # In case the study reports top hits with p < 10-6 in the GWAScatalog




In [11]:
head(GWAS_associations_df)

,association_id,pvalue,pvalue_description,pvalue_mantissa,pvalue_exponent,multiple_snp_haplotype,snp_interaction,snp_type,risk_frequency,standard_error,⋯,last_update_date,SS,ID,locus_id,variant_id,risk_allele,genome_wide,limited_list,chromosome_name,chromosome_position
,<chr>,<dbl>,<chr>,<int>,<int>,<lgl>,<lgl>,<chr>,<dbl>,<dbl>,⋯,<dttm>,<dbl>,<chr>,<int>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<int>
1,49128432,2e-13,NA,2,-13,FALSE,FALSE,novel,0.7762,0.0023,⋯,2025-06-14 01:10:29,806834,GCST009004,1,rs12072739,A,FALSE,FALSE,1,97850337
2,49128437,1e-12,NA,1,-12,FALSE,FALSE,novel,0.1212,0.0026,⋯,2025-06-14 01:11:07,806834,GCST009004,1,rs12098284,T,FALSE,FALSE,10,74287706
3,49128442,7e-24,NA,7,-24,FALSE,FALSE,novel,0.3355,0.0018,⋯,2025-06-14 01:11:08,806834,GCST009004,1,rs12121950,T,FALSE,FALSE,1,49244592
4,49128447,1e-25,NA,1,-25,FALSE,FALSE,novel,0.0911,0.0034,⋯,2025-06-14 01:11:08,806834,GCST009004,1,rs12140153,T,FALSE,FALSE,1,62114219
5,49128452,5e-12,NA,5,-12,FALSE,FALSE,novel,0.1115,0.0027,⋯,2025-06-14 01:11:15,806834,GCST009004,1,rs12147845,T,FALSE,FALSE,14,100678259
6,49128457,7e-27,NA,7,-27,FALSE,FALSE,novel,0.4087,0.0020,⋯,2025-06-14 01:11:16,806834,GCST009004,1,rs12151152,A,FALSE,FALSE,19,47060275


In [63]:
# Tidy-up
signals.b38 <- GWAS_associations_df %>%
                    dplyr::select("variant_id", chrom="chromosome_name", pos="chromosome_position", "ID")

# Convert to the hg build if needed
signals.hg19 <- lifOverFunction_vcf(signals.b38, ch_path=file.path(working_dir, "ReferenceData/hg38ToHg19.over.chain"))

# Tidy-up
top <- signals.hg19 %>%
        dplyr::rename("rsid"=variant_id,
              "position"=start,
              "chromosome"=seqnames) %>%
        mutate(chromosome=gsub("chr", "", chromosome)) %>%
        dplyr::filter(!is.na(position)) %>%
        dplyr::select(rsid, chromosome, position)

# Save the results
save(top, file = file.path("GWAS_sumstats", paste0(GWAS_ID,"_topHits.rda")))


In [46]:
# Load the data if already processed
#load(file = file.path("GWAS_sumstats", paste0(GWAS_ID,"_topHits.rda")))

# top_tmp <- top
# top_tmp$position2 <- top_tmp$position
# top_tmp$chr <- as.numeric(as.character(top_tmp$chr))
# top <- as.data.frame(top_tmp)


In [47]:
head(top)
dim(top)

,rsid,chromosome,position,position2,chr
,<chr>,<chr>,<int>,<int>,<dbl>
1,rs12072739,1,98315893,98315893,1
2,rs12098284,10,76047464,76047464,10
3,rs12121950,1,49710264,49710264,1
4,rs12140153,1,62579891,62579891,1
5,rs12147845,14,101144596,101144596,14


[1] 5 5

In [64]:
# Since this is an example, we will only use the top 5 hits
top <- top[1:5,]

In [65]:
############# Extract full summary stats from the windows of interest

vcfname <- paste0("coloc/VCF/BMI.vcf.bgz")


# Define the window length here
window_length= 1e5 # 100kb

# Extract the sumstats
GWAS_associations <- GWAS_sumstats_extract(vcfname, top, window_length, output_path="GWAS_sumstats")

# save
save(GWAS_associations, file = file.path("GWAS_sumstats/BMI.rda"))


[1] "Locus 1 of 5"
[1] "Locus 2 of 5"
[1] "Locus 3 of 5"
[1] "Locus 4 of 5"
[1] "Locus 5 of 5"


In [42]:
# Load the data if already processed
# load(file = file.path("GWAS_sumstats/BMI.rda"))

In [17]:
head(GWAS_associations)

chr,position,beta,se,p,n,id,rsid,ea,nea,eaf
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,98216085,0.0108,0.0041,8.289914e-03,806834,a,rs74108304,T,C,0.0567
1,98218276,-0.0007,0.0022,7.558994e-01,806834,a,rs56202188,A,C,0.2751
1,98218887,0.0099,0.0041,1.472007e-02,806834,a,rs74108305,T,C,0.0581
1,98220320,-0.0127,0.0031,5.226970e-05,806834,a,rs56012174,C,G,0.8959
1,98227080,-0.0105,0.0024,9.646949e-06,806834,a,rs61786604,A,G,0.7976
1,98230887,-0.0089,0.0021,2.577983e-05,806834,a,rs17378671,T,C,0.8012


In [19]:
dir.create("coloc/QTL_sumstats")

In [66]:
############### Now we create a window around the QTL lead variants and find the QTL windows that overlap the GWAS windows
# 20 peer factors + 5 PCA
print(paste0("Loading QTL and creating window around lead variant for ", GWAS_ID))
PATH_IN_M = "/lustre/groups/itg/teams/zeggini/projects/child_diabesity/analysis/methyl/fastQTL/Results/peer20_PCA5/Results/"


# Get the most significant SNP per methylation site 
mqtl <- fread(file.path(PATH_IN_M, "fastqtl_perm_ALLchr.genes.txt.gz"), header=TRUE, data.table=FALSE, tmpdir="temp") # gene_id, variant_id, qval

# Filter for qval < 0.05
mqtl <- mqtl[mqtl$qval < 0.05,]

mqtl_g <- left_join(mqtl,
                    SNP_POS,
                    by = c("variant_id"="ID"))


# Create QTL windows 
 mqtl_g <- mqtl_g %>%
                mutate(mqtl_start_coord = pos - window_length,
                mqtl_end_coord = pos + window_length)
    
    
 # Find overlap between the QTL and GWAS windows
top_tmp <- top
top_tmp$position2 <- top_tmp$position
top_tmp$chr <- as.numeric(as.character(top_tmp$chr))

setDT(mqtl_g)  
setDT(top_tmp)  
setkey(mqtl_g, "chr", "mqtl_start_coord", "mqtl_end_coord")
setkey(top_tmp, "chr", "position", "position2")


mqtl_g_subset<-foverlaps(mqtl_g, top_tmp, type="any", nomatch=NULL, mult="all")

top <- as.data.frame(top_tmp)

fwrite(data.frame(CpG=unique(mqtl_g_subset$gene_id)), paste0("coloc/QTL_sumstats/mqtl_GWAS_uniqueCpG_overlap_",GWAS_ID,".txt"), sep="\t")

# Extract the full summary stats for this set of CpGs. The full data would be too big
print(paste0("Extracting full QTL sumstats in overlapping window for ", GWAS_ID))
system(paste0("zcat ", PATH_IN_M, "/fastqtl_ALLchr_qtlmsite.txt.gz | head -n1 > ",
                paste0("coloc/QTL_sumstats/mqtl_GWAS_overlappingSNPs_",GWAS_ID,".txt; "),
                "zgrep -w -f ",
                  paste0("coloc/QTL_sumstats/mqtl_GWAS_uniqueCpG_overlap_",GWAS_ID,".txt "),
                  PATH_IN_M, "/fastqtl_ALLchr_qtlmsite.txt.gz >> ",
                  paste0("coloc/QTL_sumstats/mqtl_GWAS_overlappingSNPs_",GWAS_ID,".txt")))




[1] "Loading QTL and creating window around lead variant for GCST009004"
[1] "Extracting full QTL sumstats in overlapping window for GCST009004"


In [21]:
list.files("coloc/QTL_sumstats")

[1] "mqtl_GWAS_overlappingSNPs_GCST009004.txt"  
[2] "mqtl_GWAS_uniqueCpG_overlap_GCST009004.txt"

In [ ]:
###################################################################
####  CODE FROM ANA TO EXTRACT THE SUMSTATS FROM PARQUET FILES
###################################################################

# Extract the full summary stats for this set of traits. The full data would be too big
print(paste0("Extracting full QTL sumstats in overlapping window for ",
                 length(unique(mqtl_g_subset$gene_id)),
                " traits for GWAS ID ", GWAS_ID))
      
tmp.dt <- data.table()
for(c in seq(1,22)){
    print(c)
    qtl <- as.data.table(arrow::read_parquet(paste0(PATH_IN_M, "cis_qtl_pairs.chr", c, ".parquet")))
    qtl <- qtl[phenotype_id %in% unique(mqtl_g_subset$gene_id)]
    if(nrow(qtl)==0) next
    qtl[, c("chr", "pos", "REF", "ALT", "rsid"):=tstrsplit(variant_id, "_")]
    qtl[, maf:=ifelse(af<0.5, af, 1-af)]
    qtl[, pos:=as.integer(pos)]
    qtl[, chr:=as.integer(sub("chr", "", chr))]
    qtl[, gene_id:=phenotype_id]
    qtl <- qtl[rsid!="."]
    tmp.dt <- rbind(tmp.dt, qtl)
}
rm(qtl)
fwrite(tmp.dt, paste0("coloc/QTL_sumstats/mqtl_GWAS_overlappingSNPs_",GWAS_ID,".txt"))

      

In [22]:
# Clean up and clear some memory
rm(mqtl)
rm(tmp.dt)
rm(mqtl_g_subset)
gc()
      

Warning message in rm(tmp.dt):
“object 'tmp.dt' not found”


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,81042209,4328.2,126679657,6765.5,126679657,6765.5
Vcells,781793613,5964.7,1694957434,12931.5,1399784683,10679.6


In [67]:
# prepare GWAS genomic ranges
print(paste0("Preparing GWAS genomic ranges for ", GWAS_ID))
GWAS_signals_coloc = prepare_gwas_sig(in_gwas_signals = top, 
                                           in_mb_dist = window_length)
      
      
      

[1] "Preparing GWAS genomic ranges for GCST009004"


In [68]:
# Load QTL full summary stats for the selected QTLs
mqtl_df <- fread(paste0("coloc/QTL_sumstats/mqtl_GWAS_overlappingSNPs_",GWAS_ID,".txt"), data.table=FALSE)


In [69]:
# Annotate the QTL features with genomic coordinates
mqtl_df <- mqtl_df %>%
                 left_join(.,
                 SNP_POS,
                 by = c("variant_id"="ID")) %>%
                 mutate(start = pos - window_length,
                  end = pos + window_length)

In [26]:
head(mqtl_df)

,gene_id,variant_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,PERM_num_var,⋯,PERM_qval,PERM_pval_nominal_threshold,qtl_msite,qtl_msite_variant,chr,pos,REF,ALT,start,end
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,⋯,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>
1,cg07217846,rs823385,-999309,186,231,0.401042,0.9536340,0.00427193,0.0733996,2316,⋯,3.66558e-45,0.000221246,1,0,1,48514499,C,T,48414499,48614499
2,cg07217846,rs692931,-998845,200,244,0.423611,0.1070560,0.12463100,0.0770643,2316,⋯,3.66558e-45,0.000221246,1,0,1,48514963,G,A,48414963,48614963
3,cg07217846,rs564986,-998709,186,230,0.399306,0.9846780,-0.00141191,0.0734503,2316,⋯,3.66558e-45,0.000221246,1,0,1,48515099,A,G,48415099,48615099
4,cg07217846,rs12042708,-998525,75,82,0.142361,0.0676485,-0.17963900,0.0978913,2316,⋯,3.66558e-45,0.000221246,1,0,1,48515283,C,T,48415283,48615283
5,cg07217846,rs59741549,-998455,74,81,0.140625,0.0623220,-0.18406800,0.0983195,2316,⋯,3.66558e-45,0.000221246,1,0,1,48515353,A,G,48415353,48615353
6,cg07217846,rs17104087,-998431,74,81,0.140625,0.0623220,-0.18406800,0.0983195,2316,⋯,3.66558e-45,0.000221246,1,0,1,48515377,A,G,48415377,48615377


In [27]:
head(variant_ann)

,variant,chr,pos,ref,alt,rsid,varid,consequence,consequence_category,info,⋯,p_hwe,n_called,n_not_called,n_hom_ref,n_het,n_hom_var,n_non_ref,r_heterozygosity,r_het_hom_var,r_expected_het_frequency
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,1:15791:C:T,1,15791,C,T,rs547522712,1:15791_C_T,splice_region_variant,missense,0.861678,⋯,0.5000000,361194,0,361194,0,0,0,0.00000e+00,NA,0.00000e+00
2,1:69487:G:A,1,69487,G,A,rs568226429,1:69487_G_A,missense_variant,missense,0.956975,⋯,0.5000040,361194,0,361190,4,0,4,1.10744e-05,NA,1.10743e-05
3,1:69569:T:C,1,69569,T,C,rs2531267,1:69569_T_C,missense_variant,missense,0.831664,⋯,0.5063150,361194,0,361058,136,0,136,3.76529e-04,NA,3.76459e-04
4,1:139853:C:T,1,139853,C,T,rs533633326,1:139853_C_T,splice_region_variant,missense,0.985255,⋯,0.5000040,361194,0,361190,4,0,4,1.10744e-05,NA,1.10743e-05
5,1:692794:CA:C,1,692794,CA,C,1:692794_CA_C,1:692794_CA_C,upstream_gene_variant,non_coding,0.824483,⋯,0.0314373,361188,6,286823,70224,4141,74365,1.94425e-01,16.9582,1.93734e-01
6,1:693731:A:G,1,693731,A,G,rs12238997,1:693731_A_G,upstream_gene_variant,non_coding,0.875969,⋯,0.1641640,361192,2,281659,74692,4841,79533,2.06793e-01,15.4290,2.06315e-01


In [70]:
##################################################################################################################################################
# Allele flip: Here we use the variant_ann as reference. This needs to be the same as the one that will be used as LD reference in coloc.susie
# My LD reference is UKBB so I am flipping both the QTL and GWAS to match UKBB
####################################################################################################################################################

variant_ann$variant_id_chrpos <- paste(variant_ann$chr, variant_ann$pos, toupper(variant_ann$alt), sep="_")
     

# Check that the QTL is on the same strand ad the GWAS. If not, flip the allele and the sign of beta
mqtl_df$variant_id_chrpos = paste(mqtl_df$chr, mqtl_df$pos, mqtl_df$ALT, sep="_")
mqtl_df$variant_id_rev = paste(mqtl_df$chr, mqtl_df$pos, mqtl_df$REF, sep="_")
print(paste0("total SNPs in QTL: ", nrow(mqtl_df)))
print(paste0("N SNPs in QTL mapping to reference allele in annotation: ", sum(mqtl_df$variant_id_chrpos %in% variant_ann$variant_id_chrpos)))
print(paste0("N SNPs in QTL mapping to alternative allele in annotation: ", sum(mqtl_df$variant_id_rev %in% variant_ann$variant_id_chrpos)))
      
tmp = ifelse(mqtl_df$variant_id_chrpos %in% variant_ann$variant_id_chrpos, "orig",
                ifelse(mqtl_df$variant_id_rev %in% variant_ann$variant_id_chrpos, "rev", NA))
      
mqtl_df$varid_for_coloc = ifelse(tmp == "orig" & !is.na(tmp), mqtl_df$variant_id_chrpos,
                                     ifelse(tmp == "rev" & !is.na(tmp), mqtl_df$variant_id_rev, NA))

mqtl_df$slope = ifelse(tmp == "orig" & !is.na(tmp), mqtl_df$slope,
                            ifelse(tmp == "rev" & !is.na(tmp), (-1) * (mqtl_df$slope), NA))

mqtl_df$ALT_coloc = ifelse(tmp == "orig" & !is.na(tmp), mqtl_df$ALT,
                                 ifelse(tmp == "rev" & !is.na(tmp), mqtl_df$REF, NA))

mqtl_df$REF_coloc = ifelse(tmp == "orig" & !is.na(tmp), mqtl_df$REF,
                                 ifelse(tmp == "rev" & !is.na(tmp), mqtl_df$ALT, NA))
      
mqtl_df$ALT <- mqtl_df$ALT_coloc
mqtl_df$REF <- mqtl_df$REF_coloc
mqtl_df = mqtl_df[!is.na(mqtl_df$varid_for_coloc),]  # 158023
      
      

[1] "total SNPs in QTL: 275050"
[1] "N SNPs in QTL mapping to reference allele in annotation: 272488"
[1] "N SNPs in QTL mapping to alternative allele in annotation: 0"


In [29]:
head(mqtl_df)

,gene_id,variant_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,PERM_num_var,⋯,pos,REF,ALT,start,end,variant_id_chrpos,variant_id_rev,varid_for_coloc,ALT_coloc,REF_coloc
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,⋯,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,cg07217846,rs823385,-999309,186,231,0.401042,0.9536340,0.00427193,0.0733996,2316,⋯,48514499,C,T,48414499,48614499,1_48514499_T,1_48514499_C,1_48514499_T,T,C
2,cg07217846,rs692931,-998845,200,244,0.423611,0.1070560,0.12463100,0.0770643,2316,⋯,48514963,G,A,48414963,48614963,1_48514963_A,1_48514963_G,1_48514963_A,A,G
3,cg07217846,rs564986,-998709,186,230,0.399306,0.9846780,-0.00141191,0.0734503,2316,⋯,48515099,A,G,48415099,48615099,1_48515099_G,1_48515099_A,1_48515099_G,G,A
4,cg07217846,rs12042708,-998525,75,82,0.142361,0.0676485,-0.17963900,0.0978913,2316,⋯,48515283,C,T,48415283,48615283,1_48515283_T,1_48515283_C,1_48515283_T,T,C
5,cg07217846,rs59741549,-998455,74,81,0.140625,0.0623220,-0.18406800,0.0983195,2316,⋯,48515353,A,G,48415353,48615353,1_48515353_G,1_48515353_A,1_48515353_G,G,A
6,cg07217846,rs17104087,-998431,74,81,0.140625,0.0623220,-0.18406800,0.0983195,2316,⋯,48515377,A,G,48415377,48615377,1_48515377_G,1_48515377_A,1_48515377_G,G,A


In [71]:
# Flip the GWAS
GWAS_associations$variant_id_chrpos = paste(GWAS_associations$chr, GWAS_associations$position, GWAS_associations$ea, sep="_")
GWAS_associations$variant_id_rev = paste(GWAS_associations$chr, GWAS_associations$position, GWAS_associations$nea, sep="_")
print(paste0("total SNPs in GWAS: ", nrow(GWAS_associations)))
print(paste0("N SNPs in GWAS mapping to reference allele in annotation: ", sum(GWAS_associations$variant_id_chrpos %in% variant_ann$variant_id_chrpos)))
print(paste0("N SNPs in GWAS mapping to alternative allele in annotation: ", sum(GWAS_associations$variant_id_rev %in% variant_ann$variant_id_chrpos)))
      
tmp = ifelse(GWAS_associations$variant_id_chrpos %in% variant_ann$variant_id_chrpos, "orig",
                  ifelse(GWAS_associations$variant_id_rev %in% variant_ann$variant_id_chrpos, "rev", NA))
      
GWAS_associations$varid_for_coloc = ifelse(tmp == "orig" & !is.na(tmp), GWAS_associations$variant_id_chrpos,
                                               ifelse(tmp == "rev" & !is.na(tmp), GWAS_associations$variant_id_rev, NA))
GWAS_associations$beta = ifelse(tmp == "orig" & !is.na(tmp), GWAS_associations$beta,
                                    ifelse(tmp == "rev" & !is.na(tmp), (-1) * (GWAS_associations$beta), NA))
GWAS_associations$ea_coloc = ifelse(tmp == "orig" & !is.na(tmp), GWAS_associations$ea,
                                       ifelse(tmp == "rev" & !is.na(tmp), GWAS_associations$nea, NA))
GWAS_associations$nea_coloc = ifelse(tmp == "orig" & !is.na(tmp), GWAS_associations$nea,
                                        ifelse(tmp == "rev" & !is.na(tmp), GWAS_associations$ea, NA))
      
      
GWAS_associations$ea <- GWAS_associations$ea_coloc
GWAS_associations$nea <- GWAS_associations$nea_coloc
GWAS_associations = GWAS_associations[!is.na(GWAS_associations$varid_for_coloc),]
      
save(GWAS_associations, file = file.path("GWAS_sumstats", paste0(GWAS_ID,"_GWAS_associations.rda")))
      

[1] "total SNPs in GWAS: 1464"
[1] "N SNPs in GWAS mapping to reference allele in annotation: 823"
[1] "N SNPs in GWAS mapping to alternative allele in annotation: 636"


In [72]:
########################
# Generate genomic ranges for QTL
########################
dir.create(paste0("coloc/Coloc_sumstats/"))
dir.create(paste0("coloc/Coloc_sumstats/Coloc_", GWAS_ID))
dir.create(paste0("coloc/Coloc_sumstats/Coloc_mQTL_", GWAS_ID))

cpg_df <- mqtl_g[mqtl_g$gene_id %in% unique(mqtl_df$gene_id), c("gene_id", "chr", "pos")]
cpg_df$id = cpg_df$gene_id
cpg_df$start = cpg_df$pos
cpg_df$end = cpg_df$start
cpg_df$chr <- gsub("chr", "", cpg_df$chr)
cpg_df$pos <- cpg_df$start
      
cpg_df_gr = makeGRangesFromDataFrame(cpg_df,
                                     keep.extra.columns=FALSE,
                                     ignore.strand=FALSE,
                                     seqinfo=NULL,
                                     seqnames.field=c("chr"),
                                     start.field="start",
                                     end.field=c("end"),
                                     strand.field="strand",
                                     starts.in.df.are.0based=FALSE)                                           
      
      
      
save(cpg_df_gr, file = paste0("coloc/Coloc_sumstats/mqtl_df_", GWAS_ID,".rda"))
save(mqtl_df, file = paste0("coloc/Coloc_sumstats/cpg_df_gr_", GWAS_ID,".rda"))
      
      
print(paste0("Making QTL-GWAS overlap file for ", GWAS_ID))
overlap_df = get_overlap_df_small(in_signals_gr = GWAS_signals_coloc$signals_gr, 
                                    in_cpg_df_gr = cpg_df_gr, 
                                    in_cpg_df = cpg_df, 
                                    in_peer_dat = mqtl_df, 
                                    in_gwas_sig = GWAS_signals_coloc$gwas_sigs)
      

dir.create(file.path("coloc/Coloc_sumstats/", paste0(GWAS_ID, "_coloc_rda_files")))
dir.create(file.path(paste0(GWAS_ID, "_coloc_rda_files")))
      
save(overlap_df, file= paste0("coloc/Coloc_sumstats/overlap_df_", GWAS_ID,".rda"))

Warning message in dir.create(paste0("coloc/Coloc_sumstats/")):
“'coloc/Coloc_sumstats' already exists”
Warning message in dir.create(paste0("coloc/Coloc_sumstats/Coloc_", GWAS_ID)):
“'coloc/Coloc_sumstats/Coloc_GCST009004' already exists”
Warning message in dir.create(paste0("coloc/Coloc_sumstats/Coloc_mQTL_", GWAS_ID)):
“'coloc/Coloc_sumstats/Coloc_mQTL_GCST009004' already exists”


[1] "Making QTL-GWAS overlap file for GCST009004"


Warning message in dir.create(file.path("coloc/Coloc_sumstats/", paste0(GWAS_ID, :
“'coloc/Coloc_sumstats//GCST009004_coloc_rda_files' already exists”
Warning message in dir.create(file.path(paste0(GWAS_ID, "_coloc_rda_files"))):
“'GCST009004_coloc_rda_files' already exists”


In [54]:
# Load if already done
#load(file= paste0("coloc/Coloc_sumstats/overlap_df_", GWAS_ID,".rda"))
#load(file = paste0("coloc/Coloc_sumstats/mqtl_df_", GWAS_ID,".rda"))
#load(file = paste0("coloc/Coloc_sumstats/cpg_df_gr_", GWAS_ID,".rda"))


In [73]:
# Set the GWAS trait type, either quantitative or case-control
type = ifelse(length(GWAS_n) == 1, "quant", "cc")

In [74]:
# Set the QTL sample size
QTL_n=288

In [75]:
# Run coloc
print(paste0("Performing coloc for ", GWAS_ID))
colocFAST_df_results = perform_coloc(overlap_df = overlap_df$overlap_df, 
                                     in_m_qtl = overlap_df$m_qtl, 
                                     out_path = working_dir, 
                                     gwas_n = GWAS_n, # If type=cc, make sure that the first number refers to the cases because coloc will use the proportion of cases
                                     in_gwas = GWAS_associations,
                                     GWAS_type=type, 
                                     GWAS_ID=GWAS_ID, 
                                     QTL_n=QTL_n, 
                                     is.eQTL=FALSE, # This is to change the columns that are reported at the end since they differ between methylation QTLs and expression QTLs 
                                     cores=5)
    
    
    


[1] "Performing coloc for GCST009004"


In [56]:
head(GWAS_associations)

chr,position,beta,se,p,n,id,rsid,ea,nea,eaf,variant_id_chrpos,variant_id_rev,varid_for_coloc,ea_coloc,nea_coloc
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,98216085,0.0108,0.0041,8.289914e-03,806834,a,rs74108304,T,C,0.0567,1_98216085_T,1_98216085_C,1_98216085_T,T,C
1,98218276,-0.0007,0.0022,7.558994e-01,806834,a,rs56202188,A,C,0.2751,1_98218276_A,1_98218276_C,1_98218276_A,A,C
1,98218887,0.0099,0.0041,1.472007e-02,806834,a,rs74108305,T,C,0.0581,1_98218887_T,1_98218887_C,1_98218887_T,T,C
1,98220320,0.0127,0.0031,5.226970e-05,806834,a,rs56012174,G,C,0.8959,1_98220320_C,1_98220320_G,1_98220320_G,G,C
1,98227080,0.0105,0.0024,9.646949e-06,806834,a,rs61786604,G,A,0.7976,1_98227080_A,1_98227080_G,1_98227080_G,G,A
1,98230887,0.0089,0.0021,2.577983e-05,806834,a,rs17378671,C,T,0.8012,1_98230887_T,1_98230887_C,1_98230887_C,C,T


In [35]:
head(colocFAST_df_results[colocFAST_df_results$PP4 > 0.8,])


,cpg,cpg_pos,cpg_gene,coloc_lead_snp_chr,coloc_lead_snp_pos,coloc_lead_snp_rs,credible_set,gwas_lead_snp,gwas_lead_snp_rs,nvariants,⋯,n_gwas_variants_sign_1emins5,n_gwas_variants_sign_1emins4,PP0,PP1,PP2,PP3,PP4,test_nea,test_ea,gwas_signal
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<chr>
nsnps29,cg26836456,98393523,DPYD,1,98332523,rs61787785,"rs75641275,rs4523552,rs61787785,rs17379561,rs4378243,rs4506495",1:98315893,rs12072739,176,⋯,88,109,1.101942e-08,6.748635e-09,1.234753e-01,0.074818390,0.8017063,TRUE,TRUE,rs12072739
nsnps34,cg19889307,75970503,ADK;AP3M1,10,76018301,rs11000941,"rs11000912,rs10824102,rs10824103,rs12220238,rs11000916,rs7083534,rs10824106,rs7909915,rs6480723,rs10762582,rs11000922,rs11000924,rs11000925,rs11000930,rs10824122,rs16931294,rs10824128,rs10824129,rs10824130,rs4320897,rs11000941",10:76047464,rs12098284,283,⋯,95,113,1.093599e-53,1.839005e-07,1.119138e-47,0.187382490,0.8126173,TRUE,TRUE,rs12098284
nsnps36,cg13649133,76020585,AP3M1,10,76031721,rs11000948,"rs11000912,rs10824102,rs10824103,rs12220238,rs10824105,rs11000916,rs7083534,rs10824106,rs7909915,rs10762582,rs11000922,rs11000930,rs10824122,rs16931294,rs10824128,rs10824129,rs10824130,rs4320897,rs11000941,rs10824133,rs12217890,rs11000945,rs11000948,rs12357643,rs11000952,rs12098284,rs11000968,rs11000979,rs7073139,rs11000987,rs11000988,rs10824146,rs10824148,rs2395087,rs2894235,rs10824151,rs11001003,rs11001014,rs10824160,rs10824161,rs10824163,rs10824164,rs11001020",10:76047464,rs12098284,283,⋯,95,113,1.956864e-12,9.208630e-08,2.002563e-06,0.093330106,0.9066678,TRUE,TRUE,rs12098284
nsnps48,cg01125548,101144596,RP11-566J3.2,14,101144596,rs12147845,rs12147845,14:101144596,rs12147845,396,⋯,47,66,3.557435e-11,3.346401e-08,1.823077e-06,0.000715646,0.9992825,TRUE,TRUE,rs12147845


In [36]:
print(paste0("Writing coloc results for ", GWAS_ID))

dir.create("coloc/Coloc_results")

fwrite(colocFAST_df_results, file= file.path("coloc/Coloc_results", paste0(GWAS_ID, "_colocABF_df_results.txt")), sep="\t")


[1] "Writing coloc results for GCST009004"


In [37]:
# Clean up
rm(overlap_df)
rm(mqtl_df)
rm(cpg_df_gr)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,94841992,5065.2,182111160,9725.8,126679657,6765.5
Vcells,823425343,6282.3,1694957434,12931.5,1682725852,12838.2
